In [1]:
import pandas as pd
import numpy as np
import os
import sys

from datetime import datetime
# Make `src` dir can be imported
project_root_path = os.path.abspath(os.path.join(os.getcwd(), '..'))  # /aicup-predict-energy-generation
sys.path.append(project_root_path)


from src.utils import choose_device

import matplotlib.pyplot as plt
import seaborn as sns

from src.fe_tools import create_time_features


In [2]:
df_raw_data = pd.read_csv('../data/processed_data/combined_data.csv')
df_raw_data['datetime'] = pd.to_datetime(df_raw_data['datetime'])
# make sure the sorting is correct
df_raw_data = df_raw_data.sort_values(by=['device','datetime']).reset_index(drop=True)

df_device = choose_device(df_raw_data, 'L10')
df_device = create_time_features(df_device)
df_device = df_device.reset_index(drop=True)

In [3]:
df_device

,datetime,windspeed,pressure,temperature,humidity,sunlight,power,device,date,month,day,hour,min,week_of_year
0,2024-03-01 17:14:06,0.0,1017.48,15.59,94.30,652.92,0.12,L10,2024-03-01,3,1,17,14,9
1,2024-03-01 17:14:47,0.0,1017.48,15.66,94.04,682.50,0.12,L10,2024-03-01,3,1,17,14,9
2,2024-03-01 17:15:47,0.0,1017.47,15.74,94.10,750.00,0.14,L10,2024-03-01,3,1,17,15,9
3,2024-03-01 17:16:47,0.0,1017.46,15.78,94.09,738.33,0.14,L10,2024-03-01,3,1,17,16,9
4,2024-03-01 17:17:47,0.0,1017.49,15.80,94.08,660.83,0.12,L10,2024-03-01,3,1,17,17,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91335,2024-07-22 05:15:28,0.0,1004.27,27.36,88.72,102.50,0.00,L10,2024-07-22,7,22,5,15,30
91336,2024-07-22 05:16:28,0.0,1004.25,27.37,88.78,115.00,0.01,L10,2024-07-22,7,22,5,16,30
91337,2024-07-22 05:17:28,0.0,1004.29,27.37,88.88,129.17,0.01,L10,2024-07-22,7,22,5,17,30
91338,2024-07-22 05:18:28,0.0,1004.29,27.38,88.88,144.17,0.01,L10,2024-07-22,7,22,5,18,30


In [4]:
df_device['group'] = df_device.groupby('date').cumcount() // 3
df_test = df_device.copy()
df_test

,datetime,windspeed,pressure,temperature,humidity,sunlight,power,device,date,month,day,hour,min,week_of_year,group
0,2024-03-01 17:14:06,0.0,1017.48,15.59,94.30,652.92,0.12,L10,2024-03-01,3,1,17,14,9,0
1,2024-03-01 17:14:47,0.0,1017.48,15.66,94.04,682.50,0.12,L10,2024-03-01,3,1,17,14,9,0
2,2024-03-01 17:15:47,0.0,1017.47,15.74,94.10,750.00,0.14,L10,2024-03-01,3,1,17,15,9,0
3,2024-03-01 17:16:47,0.0,1017.46,15.78,94.09,738.33,0.14,L10,2024-03-01,3,1,17,16,9,1
4,2024-03-01 17:17:47,0.0,1017.49,15.80,94.08,660.83,0.12,L10,2024-03-01,3,1,17,17,9,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91335,2024-07-22 05:15:28,0.0,1004.27,27.36,88.72,102.50,0.00,L10,2024-07-22,7,22,5,15,30,3
91336,2024-07-22 05:16:28,0.0,1004.25,27.37,88.78,115.00,0.01,L10,2024-07-22,7,22,5,16,30,3
91337,2024-07-22 05:17:28,0.0,1004.29,27.37,88.88,129.17,0.01,L10,2024-07-22,7,22,5,17,30,3
91338,2024-07-22 05:18:28,0.0,1004.29,27.38,88.88,144.17,0.01,L10,2024-07-22,7,22,5,18,30,4


In [5]:
df_test.loc[:, 'avg_sunlight'] = df_test.groupby(['date','group'])['sunlight'].transform('mean')
df_test['residual_sunlight'] = df_test['sunlight'] - df_test['avg_sunlight']
df_test['date'] = pd.to_datetime(df_test['date'])
df_test

,datetime,windspeed,pressure,temperature,humidity,sunlight,power,device,date,month,day,hour,min,week_of_year,group,avg_sunlight,residual_sunlight
0,2024-03-01 17:14:06,0.0,1017.48,15.59,94.30,652.92,0.12,L10,2024-03-01,3,1,17,14,9,0,695.140000,-42.220000
1,2024-03-01 17:14:47,0.0,1017.48,15.66,94.04,682.50,0.12,L10,2024-03-01,3,1,17,14,9,0,695.140000,-12.640000
2,2024-03-01 17:15:47,0.0,1017.47,15.74,94.10,750.00,0.14,L10,2024-03-01,3,1,17,15,9,0,695.140000,54.860000
3,2024-03-01 17:16:47,0.0,1017.46,15.78,94.09,738.33,0.14,L10,2024-03-01,3,1,17,16,9,1,660.553333,77.776667
4,2024-03-01 17:17:47,0.0,1017.49,15.80,94.08,660.83,0.12,L10,2024-03-01,3,1,17,17,9,1,660.553333,0.276667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91335,2024-07-22 05:15:28,0.0,1004.27,27.36,88.72,102.50,0.00,L10,2024-07-22,7,22,5,15,30,3,115.556667,-13.056667
91336,2024-07-22 05:16:28,0.0,1004.25,27.37,88.78,115.00,0.01,L10,2024-07-22,7,22,5,16,30,3,115.556667,-0.556667
91337,2024-07-22 05:17:28,0.0,1004.29,27.37,88.88,129.17,0.01,L10,2024-07-22,7,22,5,17,30,3,115.556667,13.613333
91338,2024-07-22 05:18:28,0.0,1004.29,27.38,88.88,144.17,0.01,L10,2024-07-22,7,22,5,18,30,4,152.920000,-8.750000


In [6]:
df_test_1 = df_test[df_test['date'] == pd.Timestamp('2024-07-20')]
df_test_1

,datetime,windspeed,pressure,temperature,humidity,sunlight,power,device,date,month,day,hour,min,week_of_year,group,avg_sunlight,residual_sunlight
90269,2024-07-20 05:04:24,0.00,1006.05,28.17,84.31,20.00,0.00,L10,2024-07-20,7,20,5,4,29,0,24.723333,-4.723333
90270,2024-07-20 05:05:24,0.00,1006.04,28.15,84.65,24.17,0.00,L10,2024-07-20,7,20,5,5,29,0,24.723333,-0.553333
90271,2024-07-20 05:06:24,0.26,1006.06,28.13,84.81,30.00,0.00,L10,2024-07-20,7,20,5,6,29,0,24.723333,5.276667
90272,2024-07-20 05:07:24,0.17,1006.10,28.11,84.95,37.50,0.00,L10,2024-07-20,7,20,5,7,29,1,45.556667,-8.056667
90273,2024-07-20 05:08:24,0.00,1006.11,28.08,85.01,45.00,0.00,L10,2024-07-20,7,20,5,8,29,1,45.556667,-0.556667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90500,2024-07-20 08:55:24,1.48,1006.91,47.37,25.60,18236.67,329.68,L10,2024-07-20,7,20,8,55,29,77,18331.113333,-94.443333
90501,2024-07-20 08:56:24,1.57,1006.92,47.66,25.57,18349.17,336.67,L10,2024-07-20,7,20,8,56,29,77,18331.113333,18.056667
90502,2024-07-20 08:57:24,3.57,1006.92,47.36,25.30,18407.50,344.62,L10,2024-07-20,7,20,8,57,29,77,18331.113333,76.386667
90503,2024-07-20 08:58:24,2.70,1006.93,47.19,25.17,18511.67,353.49,L10,2024-07-20,7,20,8,58,29,78,18568.750000,-57.080000


In [7]:
# df_test['rolling_avg_sunlight'] = df_test['sunlight'].rolling(window=2).mean().fillna(df_test['sunlight'])
# df_test


In [9]:
df_test

,datetime,windspeed,pressure,temperature,humidity,sunlight,power,device,date,month,day,hour,min,week_of_year,group,avg_sunlight,residual_sunlight
0,2024-03-01 17:14:06,0.0,1017.48,15.59,94.30,652.92,0.12,L10,2024-03-01,3,1,17,14,9,0,695.140000,-42.220000
1,2024-03-01 17:14:47,0.0,1017.48,15.66,94.04,682.50,0.12,L10,2024-03-01,3,1,17,14,9,0,695.140000,-12.640000
2,2024-03-01 17:15:47,0.0,1017.47,15.74,94.10,750.00,0.14,L10,2024-03-01,3,1,17,15,9,0,695.140000,54.860000
3,2024-03-01 17:16:47,0.0,1017.46,15.78,94.09,738.33,0.14,L10,2024-03-01,3,1,17,16,9,1,660.553333,77.776667
4,2024-03-01 17:17:47,0.0,1017.49,15.80,94.08,660.83,0.12,L10,2024-03-01,3,1,17,17,9,1,660.553333,0.276667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91335,2024-07-22 05:15:28,0.0,1004.27,27.36,88.72,102.50,0.00,L10,2024-07-22,7,22,5,15,30,3,115.556667,-13.056667
91336,2024-07-22 05:16:28,0.0,1004.25,27.37,88.78,115.00,0.01,L10,2024-07-22,7,22,5,16,30,3,115.556667,-0.556667
91337,2024-07-22 05:17:28,0.0,1004.29,27.37,88.88,129.17,0.01,L10,2024-07-22,7,22,5,17,30,3,115.556667,13.613333
91338,2024-07-22 05:18:28,0.0,1004.29,27.38,88.88,144.17,0.01,L10,2024-07-22,7,22,5,18,30,4,152.920000,-8.750000


In [19]:
df_test['rolling_avg_sunlight'] = (
    df_test
    .groupby('date')['sunlight']
    .transform(lambda x: x.rolling(window=3).mean())
    .fillna(df_test['sunlight'])
)

In [20]:
df_test

,datetime,windspeed,pressure,temperature,humidity,sunlight,power,device,date,month,day,hour,min,week_of_year,group,avg_sunlight,residual_sunlight,rolling_avg_sunlight
0,2024-03-01 17:14:06,0.0,1017.48,15.59,94.30,652.92,0.12,L10,2024-03-01,3,1,17,14,9,0,695.140000,-42.220000,652.920000
1,2024-03-01 17:14:47,0.0,1017.48,15.66,94.04,682.50,0.12,L10,2024-03-01,3,1,17,14,9,0,695.140000,-12.640000,682.500000
2,2024-03-01 17:15:47,0.0,1017.47,15.74,94.10,750.00,0.14,L10,2024-03-01,3,1,17,15,9,0,695.140000,54.860000,695.140000
3,2024-03-01 17:16:47,0.0,1017.46,15.78,94.09,738.33,0.14,L10,2024-03-01,3,1,17,16,9,1,660.553333,77.776667,723.610000
4,2024-03-01 17:17:47,0.0,1017.49,15.80,94.08,660.83,0.12,L10,2024-03-01,3,1,17,17,9,1,660.553333,0.276667,716.386667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91335,2024-07-22 05:15:28,0.0,1004.27,27.36,88.72,102.50,0.00,L10,2024-07-22,7,22,5,15,30,3,115.556667,-13.056667,91.110000
91336,2024-07-22 05:16:28,0.0,1004.25,27.37,88.78,115.00,0.01,L10,2024-07-22,7,22,5,16,30,3,115.556667,-0.556667,102.776667
91337,2024-07-22 05:17:28,0.0,1004.29,27.37,88.88,129.17,0.01,L10,2024-07-22,7,22,5,17,30,3,115.556667,13.613333,115.556667
91338,2024-07-22 05:18:28,0.0,1004.29,27.38,88.88,144.17,0.01,L10,2024-07-22,7,22,5,18,30,4,152.920000,-8.750000,129.446667


In [16]:
(161.67 + 144.17 + 129.17)/3

145.00333333333333